<a href="https://colab.research.google.com/github/felipefernandes/lisa/blob/main/LiSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q -U google-generativeai
!pip install -q -U pathlib

In [5]:
import pathlib
import hashlib
import google.generativeai as genai
from google.colab import userdata, drive
import pandas as pd
import numpy as np
from IPython.display import display
from IPython.display import Markdown

In [8]:
# Configuração do GenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINIKEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [11]:
# Listando modelos disponíveis
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


In [16]:
# Documentos

estruturas = [
    {"nome": "Impromptu Networking", "duracao": "5-20 min", "proposito": "Compartilhe rapidamente os desafios e as expectativas, crie novas conexões"},
    {"nome": "9 Whys", "duracao": "5-20 min", "proposito": "Torne claro o propósito do trabalho juntos"},
    {"nome": "What, So What, Now What", "duracao": "15-45 min", "proposito": "Analisar juntos o progresso até então e decidir quais ajustes são necessários"},
    {"nome": "TRIZ", "duracao": "30-45 min", "proposito": "Pare atividades e comportamentos contraproducentes para abrir espaço para a inovação"},
    {"nome": "Appreciative Interviews", "duracao": "30-60 min", "proposito": "Descobrindo e aproveitando as causas-raiz do sucesso"},
    {"nome": "1-2-4-All", "duracao": "10-12 min", "proposito": "Engage todos simultaneamente na geração de perguntas, ideias e sugestões."},
    {"nome": "User Experience Fishbowl", "duracao": "25-70 min", "proposito": "Compartilhe com uma comunidade maior a capacidade obtida a partir da experiência"},
    {"nome": "15% Solutions", "duracao": "15-20 min", "proposito": "Descubra e concentre-se no que cada pessoa tem a liberdade e os recursos para fazer agora"},
    {"nome": "25-to-10 Crowdsourcing", "duracao": "20-30 min", "proposito": "Gerar e analisar rapidamente as ideias factíveis mais poderosas de um grupo"},
    { "nome": "Troika Consulting", "duracao": "15-30 min", "proposito": "Obtenha ajuda prática e criativa dos colegas imediatamente"},
    { "nome": "Conversation Café", "duracao": "35-60 min", "proposito": "Engage todos na compreensão de grandes desafios"},
    { "nome": "Min Specs", "duracao": "20-50 min", "proposito": "Especifique apenas os absolutos “Devemos” e “Não Devemos” para alcançar um propósito"},
    { "nome": "Wise Crowds", "duracao": "10-60 min. per person", "proposito": "Acesse a sabedoria do grupo inteiro em ciclos rápidos"},
    { "nome": "Wicked Questions", "duracao": "20 min", "proposito": "Articule os desafios paradoxais que um grupo deve enfrentar para ter sucesso"},
    { "nome": "Drawing Together", "duracao": "30-40 min", "proposito": "Revele ideias e caminhos para avançar através da expressão não-verbal"},
    { "nome": "Improv Prototyping", "duracao": "15-20 min. per round", "proposito": "Desenvolva soluções eficazes para desafios crônicos, enquanto se diverte seriamente"},
    { "nome": "Agreement-Certainty Matrix", "duracao": "30-45 min", "proposito": "Classifique os desafios nos domínios simples, complicado, complexo e caótico"},
    { "nome": "Shift & Share", "duracao": "35-90 min", "proposito": "Dissemine boas ideias e faça conexões informais com inovadores"},
    { "nome": "Heard, Seen, Respected", "duracao": "25 min", "proposito": "Pratique a escuta profunda e empatia com colegas"},
    { "nome": "Social Network Webbing", "duracao": "45-60 min", "proposito": "Mapeie conexões informais e decida como fortalecer a rede para alcançar um propósito"},
    { "nome": "Design Storyboards", "duracao": "25-70 min", "proposito": "Defina os elementos passo-a-passo para conduzir reuniões a finais produtivos"},
    { "nome": "Open Space", "duracao": "90 min. to 3 days", "proposito": "Libere a ação e liderança inerentes a grupos de qualquer tamanho"},
    { "nome": "Discovery & Action Dialogue", "duracao": "25-70 min", "proposito": "Descubra, invente e desencadeie soluções locais para problemas crônicos"},
    { "nome": "Integrated~Autonomy", "duracao": "60-80 min", "proposito": "Mude de soluções 'ou-um-ou-outro' para robustas soluções 'ambos-e'"},
    { "nome": "Generative Relationships", "duracao": "25 min", "proposito": "Revele padrões de relacionamento que criam surpreendente valor ou disfunções"},
    { "nome": "Critical Uncertainties", "duracao": "60-100 min", "proposito": "Desenvolva estratégias para operar em uma variedade de futuros plausíveis porém imprevisíveis"},
    { "nome": "Purpose-to-Practice", "duracao": "25-120 min", "proposito": "Desenhe os cinco elementos essenciais para uma iniciativa resiliente e duradoura"},
    { "nome": "Ecocycle Planning", "duracao": "60-95 min", "proposito": "Analise o portfólio completo de atividades e relacionamentos para identificar obstáculos e oportunidades para o progresso"},
    { "nome": "Panarchy", "duracao": "1-2 hr", "proposito": "Entenda como sistemas interconectados interagem, evoluem, disseminam inovação e transformam"},
    { "nome": "What I Need From You", "duracao": "45-70 min", "proposito": "Faça emergir as necessidades essenciais entre áreas e aceite ou rejeite pedidos de suporte"},
    { "nome": "Celebrity Interview", "duracao": "25-60 min", "proposito": "Reconectar a experiência de líderes e especialistas com pessoas mais próximas do desafio em mãos"},
    { "nome": "Helping Heuristics", "duracao": "15 min", "proposito": "Pratique métodos progressivos para ajudar os outros, receber ajuda e pedir ajuda"},
    { "nome": "Simple Ethnography", "duracao": "1-6 hr", "proposito": "Observe e registre os comportamentos reais dos usuários em campo"}
]


In [17]:
df = pd.DataFrame(estruturas)
df.columns = ['nome', 'duracao', 'proposito']
df

,nome,duracao,proposito
0,Impromptu Networking,5-20 min,Compartilhe rapidamente os desafios e as expec...
1,9 Whys,5-20 min,Torne claro o propósito do trabalho juntos
2,"What, So What, Now What",15-45 min,Analisar juntos o progresso até então e decidi...
3,TRIZ,30-45 min,Pare atividades e comportamentos contraproduce...
4,Appreciative Interviews,30-60 min,Descobrindo e aproveitando as causas-raiz do s...
5,1-2-4-All,10-12 min,Engage todos simultaneamente na geração de per...
6,User Experience Fishbowl,25-70 min,Compartilhe com uma comunidade maior a capacid...
7,15% Solutions,15-20 min,Descubra e concentre-se no que cada pessoa tem...
8,25-to-10 Crowdsourcing,20-30 min,Gerar e analisar rapidamente as ideias factíve...
9,Troika Consulting,15-30 min,Obtenha ajuda prática e criativa dos colegas i...


In [15]:
# Inicializando o modelo
model = "models/embedding-001"

In [19]:
# Função para adicionar colunas de Embeds
def addEmbeddings(nome, proposito):
  return genai.embed_content(model=model, content=proposito, title=nome, task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [20]:
# Adicionando Embeddings no DataFrame
df["Embeddings"] = df.apply(lambda row: addEmbeddings(row["nome"], row["proposito"]), axis=1)
df

In [21]:
# Função para gerar e buscar e consultar
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model, content=consulta, task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df['Embeddings']), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)

  return df.iloc[indice]['nome'] # dado um contexto, retorna o nome da EL mais adequada


In [22]:
consulta = "Time experiente, que precisa de novas idéias para um projeto"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

Improv Prototyping


In [ ]:
# Montando o Google Drive

In [ ]:
# Conteudo da aula 5, para user embeds